In [8]:
import pandas as pd
import os

In [9]:
def getXY(ids,labels,concat_cols):
    data_icu = True
    concat = False
    X_df=pd.DataFrame()   
    y_df=pd.DataFrame()   
    features=[]
    #print(ids)
    # For every one of the samples in the ./data/csv folder, get the data. 
    for sample in ids:
        # Get different labels depending on whether or not we are looking at ICU data. 
        if data_icu:
            y=labels[labels['stay_id']==sample]['label']
        else:
            y=labels[labels['hadm_id']==sample]['label']

        # Read in the dynamic data. 
        dyn=pd.read_csv('./data/csv/'+str(sample)+'/dynamic.csv',header=[0,1])

        # I do not know what this concat stuff is doing, but I believe it is not what we ae working on. 
        if concat:
            dyn.columns=dyn.columns.droplevel(0)
            dyn=dyn.to_numpy()
            dyn=dyn.reshape(1,-1)
            #print(dyn.shape)
            #print(len(concat_cols))
            dyn_df=pd.DataFrame(data=dyn,columns=concat_cols)
            features=concat_cols

        # This is the only one we are really going to use, for now. 
        else:
            dyn_df=pd.DataFrame()
            #print(dyn)
            for key in dyn.columns.levels[0]:
                #print(sample)                    
                dyn_temp=dyn[key]
                # This is the one that we are using. 
                if data_icu:
                    if ((key=="CHART") or (key=="MEDS")):
                        agg=dyn_temp.aggregate("mean")
                        agg=agg.reset_index()
                    else:
                        agg=dyn_temp.aggregate("max")
                        agg=agg.reset_index()
                else:
                    if ((key=="LAB") or (key=="MEDS")):
                        agg=dyn_temp.aggregate("mean")
                        agg=agg.reset_index()
                    else:
                        agg=dyn_temp.aggregate("max")
                        agg=agg.reset_index()
                if dyn_df.empty:
                    dyn_df=agg
                else:
                    dyn_df=pd.concat([dyn_df,agg],axis=0)
            dyn_df=dyn_df.T
            dyn_df.columns = dyn_df.iloc[0]
            dyn_df=dyn_df.iloc[1:,:]
        # get the static and demo data. 
        stat=pd.read_csv('./data/csv/'+str(sample)+'/static.csv',header=0)
        demo=pd.read_csv('./data/csv/'+str(sample)+'/demo.csv',header=0)

        # Fill up the X_df with the demo data. 
        if X_df.empty:
            X_df=pd.concat([dyn_df,stat],axis=1)
            X_df=pd.concat([X_df,demo],axis=1)
        else:
            X_df=pd.concat([X_df,pd.concat([pd.concat([dyn_df,stat],axis=1),demo],axis=1)],axis=0)

        # Just make y equal to the label (with the labels describing how many 
        if y_df.empty:
            y_df=y
        else:
            y_df=pd.concat([y_df,y],axis=0)
            
    # remove all of the duplicate columns. 
    X_df = X_df.loc[:,~X_df.columns.duplicated()].copy()
    print("X_df",X_df.shape)
    print("y_df",y_df.shape)
    return X_df ,y_df

In [10]:
all_csv = os.listdir("data/csv")
concat_cols = None
labels=pd.read_csv('./data/labels.csv', header=0)

In [11]:
ids = []
for csv in all_csv:
    try:
        ids.append(int(csv))
    except:
        continue

In [12]:
X_df, Y_df = getXY(ids,labels,concat_cols)

X_df (2538, 25)
y_df (2538,)


In [13]:
X_df

,220045,220179,220210,220277,223761,220293,224685,220224,220235,225625,...,intime,outtime,Age,gender,ethnicity,insurance,label,hadm_id,los,select_time
0,71.541667,133.833333,20.736111,93.875,24.495833,0.0,0.0,0.0,0.0,0.359722,...,2171-09-03 15:44:00,2171-09-08 17:31:24,69,F,WHITE - RUSSIAN,Other,0,23826411,72,49
0,76.925926,99.15,12.301389,87.993056,20.5,2.75,59.055556,5.930556,0.569444,0.752778,...,2156-09-30 19:08:00,2156-10-03 22:14:44,65,F,WHITE,Medicare,0,24353007,72,3
0,88.583333,100.847222,15.361111,95.361111,25.956944,2.075,79.291667,2.208333,1.597222,0.909722,...,2122-12-13 03:41:21,2122-12-16 18:29:11,57,F,WHITE,Other,0,26261796,72,14
0,81.305556,117.777778,18.875,46.930556,20.322222,0.0,0.0,0.0,0.0,0.373611,...,2119-03-26 18:53:57,2119-03-29 21:01:24,91,F,WHITE,Medicare,0,21213879,72,2
0,79.986111,28.902778,18.027778,74.486111,20.293056,0.0,0.0,0.902778,0.638889,0.476389,...,2178-09-22 09:36:13,2178-10-02 18:39:36,73,M,WHITE,Medicare,0,22793802,72,177
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,58.397685,77.29213,8.683565,59.768056,27.1875,0.0,0.0,0.0,0.0,0.384722,...,2164-07-30 17:49:50,2164-08-02 21:20:46,62,F,WHITE,Medicaid,0,24259026,72,3
0,74.041667,97.215278,17.361111,95.986111,23.036111,0.0,0.0,0.0,0.0,0.8375,...,2147-03-25 00:41:01,2147-04-03 18:12:11,57,M,WHITE,Other,0,23089468,72,161
0,80.027778,162.194444,19.229167,96.826389,25.916667,0.0,0.0,0.0,0.0,0.401389,...,2161-04-29 16:41:00,2161-05-02 17:21:43,86,F,WHITE,Medicare,0,28926211,72,0
0,56.993924,102.509921,18.157986,96.527778,24.511111,2.805556,74.298611,0.0,0.0,0.615278,...,2154-10-26 15:18:49,2154-10-29 15:57:49,77,F,BLACK/AFRICAN AMERICAN,Medicare,0,22779758,72,0


In [14]:
Y_df

604     0
1424    0
820     0
1336    0
1480    0
       ..
1399    0
744     0
542     0
1923    0
878     0
Name: label, Length: 2538, dtype: int64